# Assignment 2 preamble

Please remember to change the name of your copy of the assignment to include your name (and optionally UNI). 

We're going to try having you submit the ipynb file (File -> Download .ipynb) rather than sharing through Google drive this time. 

Most of the tasks/questions for this assignment are at the end but there is one in the middle so don't miss it! 

# Setting up Colab

1.   Make sure you're signed into Colab through your Columbia account. This will be important for accessing data. 
2.   Check you've requested a GPU (or TPU): go to Runtime -> Change runtime type -> Set "Hardware Accelerator". Colab will restart if needed. 

Mount your Google drive. This should give you a link to follow where you'll copy a key that you'll paste in here. You can view files in the sidebar on the left under "Files". 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Load `torch`. Note the version on colab can lag behind the most current release. If you need something new you can install using e.g. !pip3 install torch==1.4.0

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data
assert(torch.cuda.is_available()) # if this fails go to Runtime -> Change runtime type -> Set "Hardware Accelerator"
print("Torch version:", torch.__version__)

## Utilities

We'll be using a "one hot" encoding of DNA sequence as the input to our CNN. This can actually be a bottleneck as it runs on the CPU so we use Cython to speed things up.  

In [ ]:
%load_ext Cython

In [ ]:
%%cython

import numpy as np
np.get_include() # do we need this on colab? 
cimport cython
cimport numpy as np

cdef dict bases={ 'A':<int>0, 'C':<int>1, 'G':<int>2, 'T':<int>3 } 

@cython.boundscheck(False)
def one_hot( str string ):
    cdef np.ndarray[np.float32_t, ndim=2] res = np.zeros( (4,len(string)), dtype=np.float32 )
    cdef int j
    for j in range(len(string)):
        if string[j] in bases: # bases can be 'N' signifying missing: this corresponds to all 0 in the encoding
            res[ bases[ string[j] ], j ]=float(1.0)
    return(res)


# Pytorch basics

`pytorch` (as opposed to e.g. Theano, Tensorflow 1) uses *eager execution*: this lets you write computations as python code that you can test and debug, and later 
1.   Backprop through (i.e. get gradients with respect to inputs) 
2.   Run on the GPU for (hopefully!) big speedups. 

Here's an example:

In [ ]:
x_np = one_hot("CCGCGNGGNGGCAG")
x_tensor = torch.tensor(x_np)
print(x_tensor)
torch.sum(x_tensor, 1)

`torch` has equivalents for most `numpy` operations, for example here we got the row sums to check how often each base appeared in our sequence. 

As a more exciting example, `torch.nn.functional` has convolutions implemented for us. 

In [ ]:
import torch.nn.functional as F
help(F.conv1d)

Notice `conv1d` expects
1. `input` to have shape batchsize x #input_channels x length (as a warning, `tensorflow` expects batchsize x length x #input_channels!)
2. `weight` (i.e. the filters) to have shape #output_channels x #input_channels x filter_width. 

We can make a random tensor for the filters (weights), and then execute the convolution 

In [ ]:
x_batch = x_tensor[None,:,:] # make a "batch" of size 1
filter_width = 5 
my_weights = torch.randn(1, 4, filter_width) # 1 output channels, 4 input channels, filter width = 5
convolution_output = F.conv1d(x_batch, my_weights)
convolution_output

Let's check `torch` got the right answer. Here's the calculation for the first position (for data point 0 and output channel 0): 

In [ ]:
(x_batch[0,:,0:filter_width] * my_weights[0,:,:]).sum()

## Task

**Write a for loop to manually calculate the remaining terms in the convolution [1 point]**

In [ ]:
my_convolution_output = torch.zeros([1, 1, x_batch.shape[2] - (filter_width-1)])
for i in range(my_convolution_output.shape[2]): 
    my_convolution_output[0,0,i] = # TODO YOUR CODE HERE
assert( ((convolution_output - my_convolution_output).abs() < 1e-6).all() )

Note the effect of the (0 padded) convolution on the shape of the output: the sequence length has gone from 14 to 10. 

In [ ]:
print( x_batch.shape, convolution_output.shape )

Without padding we lose (filter_width-1) positions. 

`torch` has nice built in utilities for setting up neural network (NN) layers. For example, we can make a 1D convolutional layer like this: 

In [ ]:
torch.manual_seed(2) # I played with different initialization here! 
my_first_conv_layer = nn.Conv1d(4, 1, 14, padding = 0) # 4 input channels, 1 output channels, filter width 14, no padding
my_first_conv_layer

We can use this layer like a function (applied to tensors): 

In [ ]:
my_first_conv_layer(x_batch)

But if `my_first_conv_layer` behaves like a function, where are the weights? They're there (along with biases), but they're a little hidden: 

In [ ]:
list(my_first_conv_layer.parameters())

But where did these numbers come from? `torch` by default initializes using the very reasonable [Kaiming/He initalization](https://arxiv.org/abs/1502.01852) which aims to keep the variance of the hidden layer activations reasonably stable (constant in expectation) through the network. 

# My First Convolutional Neural Network

We're ready to make the simplest CNN possible where we just take the max over all the convolution outputs: 

In [ ]:
def my_simplest_CNN(x): 
    net = my_first_conv_layer(x)
    net = net[:,0,:] # only one output channel! 
    # take maximum over channel ("global max pooling")
    net = torch.max(net, dim=1).values # max returns namedtuple (values, indices)
    net = torch.sigmoid(net) # aka logistic to get output in [0,1]
    return(net) 

my_simplest_CNN(x_batch)

## Loading data

To test this out we need some data! Our first task will be predict binding of the important transcriptional repressor CTCF in a human lung cancer cell line called A549. The data is available from ENCODE including merging replicate experiments using the "irreproducible discovery rate" (IDR) [paper](https://arxiv.org/abs/1110.4705) [code](https://github.com/spundhir/idr). Genomics data representing discrete binding events is typically stored in the  `bed` format, which is described on the UCSC Genome Browser [website](https://genome.ucsc.edu/FAQ/FAQformat.html#format1). This bed file has some additional columns we will ignore. 

In [ ]:
import pandas as pd
DATADIR = "/content/drive/My Drive/ML4fungen/Assignment 2/" # might need to change this
binding_data = pd.read_csv(DATADIR + "ENCFF300IYQ.bed.gz", sep='\t', usecols=range(6), names=("chrom","start","end","name","score","strand"))
binding_data = binding_data[ ~binding_data['chrom'].isin(["chrX","chrY"]) ] # only keep autosomes (non sex chromosomes)
binding_data = binding_data.sort_values(['chrom', 'start']).drop_duplicates() # sort so we can interleave negatives
binding_data[:10]

`chrom` is the chromosome (we filter out the sex chromosomes X and Y to avoid bias from these being haploid, you could try keeping them though). `start` and `end` are positions in the genome. `name` and `strand` aren't used here. Some genomics assays (e.g. RNA-seq) correspond to one strand of the DNA or the other. However, for our purposes at least we can consider ChIP-seq and ATAC-seq as "unstranded", which is why that column is all "." rather than + or -.  `score` is a quantitative measure of binding. We'll ignore this for now (since all these peaks are statistically significant), but you could try to incorporate it in your model training if you want (e.g. multitask for binary binding plus predicting `score`). 

We'll split the binding data into training, validation (chroms 2 and 3, ~) and test (chrom 1), which represents about 14% of the training data we have: 

In [ ]:
test_chromosomes = ["chr1"]
test_data = binding_data[ binding_data['chrom'].isin( test_chromosomes ) ]

validation_chromosomes = ["chr2","chr3"]
validation_data = binding_data[ binding_data['chrom'].isin(validation_chromosomes) ]

train_chromosomes = ["chr%i" % i for i in range(4, 22+1)]
train_data = binding_data[ binding_data['chrom'].isin( train_chromosomes ) ]

test_data.shape[0] / binding_data.shape[0], validation_data.shape[0] / binding_data.shape[0]

We'll also need the human genome, which we provide here as a pickle since it's faster to load compared to reading in a text file. 

It's worth knowing that the human genome has different *versions* that are released as more missing parts are resolved by continued sequencing and assembly efforts. Version `GRCh37` (also called `hg19`) was released in 2009, and `GRCh38` (`hg38`) was released in 2013. We'll be using `hg19` here but `GRCh38` is finally becoming more standard so always check your data is what you think it is. 

This will take a minute or two. 

In [ ]:
import pickle
#genome = pickle.load(open(DATADIR+"hg38.pkl","rb")) # this is here in case there's hg38 data you want to analyse
genome = pickle.load(open(DATADIR+"hg19.pickle","rb"))

`genome` is just a dictionary where the keys are the chromosome names and the values are strings representing the actual DNA: 

In [ ]:
genome["chr13"][100000000:100000010]

You'll find a substantial proportion of each chromosome is "N"s: 

In [ ]:
genome["chr13"].count("N") / len(genome["chr13"])

Ns represents "missing" regions, typically because the region has too many repetitive sequences making mapping impossible, which is especially the case in [centrosomes](https://en.wikipedia.org/wiki/Centrosome) and [telomeres](https://en.wikipedia.org/wiki/Telomere). Resolving these difficult to map regions is an ongoing effort. 

We'll use the `torch` data loading utilities (nicely documented [here](https://pytorch.org/docs/stable/data.html)) to handle
1. Grouping individual (x,y) pairs into minibatches. 
2. Converting `numpy` arrays into `torch` tensors. 

We could also use `num_workers>0` to have a background process generating the next batch using the CPU while the GPU is working, but in my experience this actually slows things down. If you were in a computer vision setting where you wanted to do intensive [data augmentation](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html) it might make a difference. 

In [ ]:
# positive example: binding of protein onto sequence (ChIP-seq (TF ChIP-seq))
# negative example: the ones that do not overlap with the positive examples 
# for chip-seq data: also shuffling nucleotides can be done to keep the GC content the same as positive example
# because sequencing has biases with GC content and this would be a way to "fix it"
class BedPeaksDataset(torch.utils.data.IterableDataset):

    def __init__(self, atac_data, genome, context_length):
        super(BedPeaksDataset, self).__init__()
        self.context_length = context_length
        self.atac_data = atac_data
        self.genome = genome

    def __iter__(self): 
        prev_end = 0
        prev_chrom = ""
        for i,row in enumerate(self.atac_data.itertuples()):
            midpoint = int(.5 * (row.start + row.end))
            seq = self.genome[row.chrom][ midpoint - self.context_length//2:midpoint + self.context_length//2]
            yield(one_hot(seq), np.float32(1)) # positive example

            if prev_chrom == row.chrom and prev_end < row.start: 
                midpoint = int(.5 * (prev_end + row.start))
                seq = self.genome[row.chrom][ midpoint - self.context_length//2:midpoint + self.context_length//2]
                yield(one_hot(seq), np.float32(0)) # negative example midway inbetween peaks, could randomize
            
            prev_chrom = row.chrom
            prev_end = row.end

train_dataset = BedPeaksDataset(train_data, genome, 100)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1000, num_workers = 0)

train_data

## Training

Let's train our super simple CNN using stochastic gradient descent: 

In [ ]:
import timeit
start_time = timeit.default_timer()

torch.set_grad_enabled(True) # we'll need gradients

for epoch in range(10): # run for this many epochs
    losses = []
    accuracies = []
    for (x,y) in train_dataloader: # iterate over minibatches

        output = my_simplest_CNN(x) # forward pass
        # in practice (and below) we'll use more numerically stable built-in
        # functions for the loss
        loss = - torch.mean( y * torch.log(output) + (1.-y) * torch.log(1.-output) )
        loss.backward() # back propagation

        # iterate over parameter tensors: just the layer1 weights and bias here
        for parameters in my_first_conv_layer.parameters(): 
            parameters.data -= 1.0 * parameters.grad # in practive reduce or adapt learning rate
            parameters.grad.data.zero_() # torch accumulates gradients so need to reset
        
        losses.append(loss.detach().numpy()) # convert back to numpy
        accuracy = torch.mean( ( (output > .5) == (y > .5) ).float() )
        accuracies.append(accuracy.detach().numpy())  

    elapsed = float(timeit.default_timer() - start_time)
    print("Epoch %i %.2fs/epoch Loss: %.4f Acc: %.4f" % (epoch+1, elapsed/(epoch+1), np.mean(losses), np.mean(accuracies)))


So we can get pretty decent accuracy even with this very simple CNN (although I cheated a bit by trying a few random seeds and knowing that the CTCF consensus motif is 14nt long so would require a width 14 filter). 

In [ ]:
validation_dataset = BedPeaksDataset(validation_data, genome, 100)
validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=1000)
accuracies = [ torch.mean( ( (my_simplest_CNN(x)  > .5) == (y > .5) ).float() ).detach().cpu().numpy() for (x,y) in validation_dataloader ]
np.mean(accuracies)

The validation accuracy is very similiar to the train accuracy, suggesting we're not overfitting (as we would expect with such a simple model). 

## Model interpretation

Since this model used only a single convolutional filter it's very easy to interpret how it's making predictions. In genetics it's common to plot position weight matrices as "sequence logos": 

In [ ]:
!pip install logomaker
import logomaker
pwm = my_first_conv_layer.weight.detach().cpu().numpy().squeeze()
pwm -= pwm.mean(0, keepdims=True) # remove spurious degrees of freedom
pwm_df = pd.DataFrame(data = pwm.transpose(), columns=("A","C","G","T"))
crp_logo = logomaker.Logo(pwm_df) # CCACCAGG(G/T)GGCG

Characters above (below) the x-axis correspond to positive (negative) values. Our model has some additional degrees of freedom (we could add a constant k to each column of the filter and substract k from the bias without changing the prediction) which we remove. Compare this to the "known" sequence motif: [JASPAR CTCF](http://jaspar.genereg.net/matrix/MA0139.1/) to see it's a pretty good match. 

# A full LeNet-type CNN

Let's see if we can do better with a full CNN. While it's not strictly necessary it's convenient to encapsulate the model as a class inheriting from `nn.Module`. This allows automatic handling of things like: 
1. Extracting all model parameters. 
2. Setting all layers to train or eval mode (for layers like dropout that behave differently in the two settings). 
Note that here we used the model architecture to specify the length of sequence (the "sequence context") that is considered (stored as cnn_1d.seq_len). "num_chunks" here corresponds to the sequence length of the hidden layer after the last convolutional layer. The other common approach is to instead specify the sequence context and use the formula we covered in class to calculate the sequence length L for each each hidden layer (you're welcome to try implementing that approach!) 

In [ ]:
class CNN_1d(nn.Module):

    def __init__(self, 
                 n_output_channels = 1, 
                 filter_widths = [15, 5], 
                 num_chunks = 5, 
                 max_pool_factor = 4, 
                 nchannels = [4, 32, 32],
                 n_hidden = 32, 
                 dropout = 0.2):
        
        super(CNN_1d, self).__init__()
        self.rf = 0 # running estimate of the receptive field
        self.chunk_size = 1 # running estimate of num basepairs corresponding to one position after convolutions

        conv_layers = []
        for i in range(len(nchannels)-1):
            conv_layers += [ nn.Conv1d(nchannels[i], nchannels[i+1], filter_widths[i], padding = 0),
                        nn.BatchNorm1d(nchannels[i+1]), # tends to help give faster convergence: https://arxiv.org/abs/1502.03167
                        nn.Dropout2d(dropout), # popular form of regularization: https://jmlr.org/papers/v15/srivastava14a.html
                        nn.MaxPool1d(max_pool_factor), 
                        nn.ELU(inplace=True)  ] # popular alternative to ReLU: https://arxiv.org/abs/1511.07289
            assert(filter_widths[i] % 2 == 1) # assume this
            self.rf += (filter_widths[i] - 1) * self.chunk_size
            self.chunk_size *= max_pool_factor

        # If you have a model with lots of layers, you can create a list first and 
        # then use the * operator to expand the list into positional arguments, like this:
        self.conv_net = nn.Sequential(*conv_layers)

        self.seq_len = num_chunks * self.chunk_size + self.rf # amount of sequence context required

        print("Receptive field:", self.rf, "Chunk size:", self.chunk_size, "Number chunks:", num_chunks)

        self.dense_net = nn.Sequential( nn.Linear(nchannels[-1] * num_chunks, n_hidden),
                                        nn.Dropout(dropout),
                                        nn.ELU(inplace=True), 
                                        nn.Linear(n_hidden, n_output_channels) )

    def forward(self, x):
        net = self.conv_net(x)
        net = net.view(net.size(0), -1)
        net = self.dense_net(net)
        return(net)

cnn_1d = CNN_1d()

print("Input length:", cnn_1d.seq_len)

cnn_1d

The `torch` output nicely illustrates the various layers in our network. We have two convolutional units each doing conv -> batchnorm -> dropout -> maxpooling -> activation, followed by two layers of dense network, including a dropout layer.

We'll recreate the dataloaders to satisfy the input sequence length requirement of the new model. 


In [ ]:
train_dataset = BedPeaksDataset(train_data, genome, cnn_1d.seq_len)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1000, num_workers = 0)
validation_dataset = BedPeaksDataset(validation_data, genome, cnn_1d.seq_len)
validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=1000)


## Transfering to the GPU
The previous example actually ran on the CPU. Now we're using a more complex model it'll be worth running on the GPU. To do that we need to transfer both model and data. Transfering the model: 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cnn_1d.to(device)
print(device)

Check that this says `cuda` not `cpu`! You need to go to Runtime -> Change runtime type -> Set "Hardware Accelerator" -> GPU (or TPU) if not. 

Next we'll set up the optimizer. `torch` has a number of [optimizers](https://pytorch.org/docs/stable/optim.html), most of which are variants of SGD. [Adam](https://arxiv.org/abs/1412.6980) with the [AMSgrad convergence fix](https://openreview.net/forum?id=ryQu7f-RZ) is a good default in my experience. 

In [ ]:
optimizer = torch.optim.Adam(cnn_1d.parameters(), amsgrad=True)

We define a training loop which can be used for both training and validation loops by setting the `train_flag`. 

In [ ]:
def run_one_epoch(train_flag, dataloader, cnn_1d, optimizer, device="cuda"):

    torch.set_grad_enabled(train_flag)
    cnn_1d.train() if train_flag else cnn_1d.eval() 

    losses = []
    accuracies = []

    for (x,y) in dataloader: # collection of tuples with iterator

        (x, y) = ( x.to(device), y.to(device) ) # transfer data to GPU

        output = cnn_1d(x) # forward pass
        output = output.squeeze() # remove spurious channel dimension
        loss = F.binary_cross_entropy_with_logits( output, y ) # numerically stable

        if train_flag: 
            loss.backward() # back propagation
            optimizer.step()
            optimizer.zero_grad()

        losses.append(loss.detach().cpu().numpy())
        accuracy = torch.mean( ( (output > .5) == (y > .5) ).float() )
        accuracies.append(accuracy.detach().cpu().numpy())  
    
    return( np.mean(losses), np.mean(accuracies) )

## Training
Ok let's train! We'll keep track of validation loss to do [early stopping](https://en.wikipedia.org/wiki/Early_stopping): if we don't see any improvement in the validation validation loss for 10 consecutive epochs then we stop training. This will take a few minutes. 

In [ ]:
train_accs = []
val_accs = []
patience = 10 # for early stopping
patience_counter = patience
best_val_loss = np.inf
check_point_filename = 'cnn_1d_checkpoint.pt' # to save the best model fit to date
for epoch in range(100):
    start_time = timeit.default_timer()
    train_loss, train_acc = run_one_epoch(True, train_dataloader, cnn_1d, optimizer, device)
    val_loss, val_acc = run_one_epoch(False, validation_dataloader, cnn_1d, optimizer, device)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    if val_loss < best_val_loss: 
        torch.save(cnn_1d.state_dict(), check_point_filename)
        best_val_loss = val_loss
        patience_counter = patience
    else: 
        patience_counter -= 1
        if patience_counter <= 0: 
            cnn_1d.load_state_dict(torch.load(check_point_filename)) # recover the best model so far
            break
    elapsed = float(timeit.default_timer() - start_time)
    print("Epoch %i took %.2fs. Train loss: %.4f acc: %.4f. Val loss: %.4f acc: %.4f. Patience left: %i" % 
          (epoch+1, elapsed, train_loss, train_acc, val_loss, val_acc, patience_counter ))

Rather than trying to stare at those numbers let's plot training and validation accuracy: 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_accs, label="train")
plt.plot(val_accs, label="validation")
plt.legend() 
plt.grid(which="both")

Unusually the validation accuracy is higher than the train accuracy in some places! This could be due to (at least) two factors: 
1. Relatively small validation set so the estimate is noisy. 
2. Dropout introduces noise into the predictions during training but not validation (as it should). 
We could actually test this by evaluating on the training data without dropout: 

In [ ]:
train_loss, train_acc = run_one_epoch(False, train_dataloader, cnn_1d, optimizer, device)
val_loss, val_acc = run_one_epoch(False, validation_dataloader, cnn_1d, optimizer, device)
print(train_acc, val_acc)

As expected, the validation accuracy is a little lower. Strictly speaking we should also check performance on the test set since we used the validation set to do early stopping. 

In [ ]:
test_dataset = BedPeaksDataset(test_data, genome, cnn_1d.seq_len)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1000)
test_loss, test_acc = run_one_epoch(False, test_dataloader, cnn_1d, optimizer, device)
test_acc

Nice, the test set accuracy is high too. 

## Model interpretation

Model interpretation is an active area of research for CNNs. Two baseline approaches in genomics are *in silico* mutagenesis and saliency maps. Good approaches exist for explaining the prediction for a single instance: explaining how the model works globally is still a challenge in general. 

### in silico mutatgenesis

This approach is specific to genomics where we can imagine making individual "point" mutations (changing just one base) and seeing what effect that has on the model's prediction. In computer vision there isn't a direct analogy: deleting or changing a single pixel would rarely (if ever) change the prediction we would expect (although CNNs are not necessarily robust to such changes - adversarial training attempts to make them so).  

*in silico* mutagenesis can be used both for model interpretation (as we do here) and predicting the effects of real mutations/genetic differences between individuals. 

We'll just run mutagenesis for the first batch of the validation data: 

In [ ]:
torch.set_grad_enabled(False)
for (x_cpu,y_cpu) in validation_dataloader: 
    x = x_cpu.to(device)
    y = y_cpu.to(device)
    output = cnn_1d(x).squeeze()
    output = torch.sigmoid(output)
    delta_output = torch.zeros_like(x, device=device)
    # loop over all positions changing to each position nucleotide
    # note everything is implicitly parallelized over the batch here
    for seq_idx in range(cnn_1d.seq_len): # iterate over sequence
        for nt_idx in range(4): # iterate over nucleotides
            x_prime = x.clone() # make a copy of x
            x_prime[:,:,seq_idx] = 0. # change the nucleotide to nt_idx
            x_prime[:,nt_idx,seq_idx] = 1.
            output_prime = cnn_1d(x_prime).squeeze()
            output_prime = torch.sigmoid(output_prime)
            delta_output[:,nt_idx,seq_idx] = output_prime - output
    break # just do this for first batch

Note how computationally expensive this is: for every instance we do inference $4 \times L$ times (we could make this $3 \times L$ easily enough). 

We'll visualize just four (2 positive 2 negative) examples: 

In [ ]:
delta_output_np = delta_output.detach().cpu().numpy()
delta_output_np -= delta_output_np.mean(1, keepdims=True)
output_np = output.detach().cpu().numpy()
plt.figure(figsize = (12,12))
for i in range(1,5):
    ax = plt.subplot(4,1,i)
    pwm_df = pd.DataFrame(data = delta_output_np[i,:,:].transpose(), columns=("A","C","G","T"))
    crp_logo = logomaker.Logo(pwm_df, ax = ax) # CCGCGNGGNGGCAG or CTGCCNCCNCGCGG
    plt.title("True label: %i. Prob(y=1)=%.3f" % (y_cpu[i],output_np[i]))

plt.subplots_adjust(hspace = 0.3)

Note the difference in the y-axis scales! For the two positive examples there are clear regions in the sequence that if disrupted significantly impact the prediction. For the negative examples there is a scattering of "mutations" that would effect the prediction, presumably by randomly introducing a sequence that looks a little like the CTCF motif (we don't expect any specific features in these negative sequences). 

### Saliency maps

This is the simplest approach to leverage the same backprop machinery that we use during training. The trick is that instead of taking the gradient w.r.t. parameters we'll now take the gradient w.r.t. inputs `x`. Specifically here we'll get the gradient of $P(y=1|x)$ w.r.t x. The `saliency` itself is defined as that gradient elementwise multiplied with `x` itself. 

In [ ]:
torch.set_grad_enabled(True)
x.requires_grad_() # tell torch we will want gradients wrt x (which we don't normally need)
output = cnn_1d(x).squeeze()
output = torch.sigmoid(output)
dummy = torch.ones_like(output) # in a multiclass model this would be a one-hot encoding of y
output.backward(dummy) # to get derivative wrt to x
gradient_np = x.grad.detach().cpu().numpy()
output_np = output.detach().cpu().numpy()
saliency = gradient_np * x_cpu.numpy()
plt.figure(figsize = (12,12))
for i in range(1,5):
    ax = plt.subplot(4,1,i) #,sharey=ax)
    pwm_df = pd.DataFrame(data = saliency[i,:,:].transpose(), columns=("A","C","G","T"))
    logomaker.Logo(pwm_df, ax=ax) # CCGCGNGGNGGCAG or CTGCCNCCNCGCGG
    plt.title("True label: %i. Prob(y=1)=%.3f" % (y_cpu[i],output_np[i]))

plt.subplots_adjust(hspace = 0.3)

This is much more computationally efficient than in silico mutagenesis, requiring only ONE forward and backward pass. Compare the two interpretation methods: for the positive examples the same sequence regions are highlighted (although the saliency map is noisier outside that region). 

### Other interpretation approaches

There's a nice compedium and associated `torch` code for a number methods [here](https://github.com/utkuozbulak/pytorch-cnn-visualizations). Some that I would add: 
1. DeepLIFT  [paper](https://arxiv.org/abs/1704.02685) [code](https://github.com/kundajelab/deeplift) - no `pytorch` support sadly. 
2. DeepSHAP [paper](https://papers.nips.cc/paper/7062-a-unified-approach-to-interpreting-model-predictions) [code](https://github.com/slundberg/shap) 
3. Influence functions. [paper](https://arxiv.org/abs/1703.04730) [code](https://github.com/kohpangwei/influence-release). A little different: finds which training points most strongly influence the current prediction. 

## A more challenging task

CTCF binding is relatively easy to predict, mostly depending on homotypic binding with a strong motif. Here you'll attempt a more difficult task: predicting chromatin accessibility (as measured by ATAC-seq) for a mystery cell type (mysterious so you can fairly compete in the class competition below!) Let's load data and split into training and validation - I've already removed chromosome 1 and 2 data as test set to be used in the competition. 

In [ ]:
atac_data = pd.read_csv(DATADIR + "ATAC_data.bed.gz", sep='\t', names=("chrom","start","end"))
atac_data = atac_data.sort_values(['chrom', 'start']) # actually already sorted but why not

validation_chromosomes = ["chr3","chr4"]
validation_data = atac_data[ atac_data['chrom'].isin(validation_chromosomes) ]

train_data = atac_data[ ~atac_data['chrom'].isin( validation_chromosomes ) ]

train_data.shape[0] / atac_data.shape[0], validation_data.shape[0] / atac_data.shape[0]

# Tasks

Make your own copy of this notebook complete the following questions by filling in the code and write-up sections. Feel free to add cells as needed.

## Wrapper function [2 points]

It will be helpful to make a wrapper function that does the following: 
1. Make new `BedPeakDataset` and `DataLoader` objects for both training and validation data. 
2. Instantiates an optimizer for the model. 
3. Runs the training loop with early stopping. 
4. Returns the fitted model, train and validation accuracies.

We've given a suggested signature for the wrapper function. You'll find the code snippets you need above although you will need to do some minor editing if you want the optional arguments to have the correct effects. 

In [ ]:
# TODO make wrapper function. 

def train_model(cnn_1d, train_data, validation_data, epochs=100, patience=10, verbose = True):
    """
    Train a 1D CNN model and record accuracy metrics.
    """
    # Move the model to the GPU here to make it runs there, and set "device" as above
    # TODO CODE

    # 1. Make new BedPeakDataset and DataLoader objects for both training and validation data.
    # TODO CODE

    # 2. Instantiates an optimizer for the model. 
    # TODO CODE

    # 3. Run the training loop with early stopping. 
    # TODO CODE

    # 4. Return the fitted model (not strictly necessary since this happens "in place"), train and validation accuracies.
    # TODO CODE

You should now be able to train a basic CNN with the same architechure as we used above for CTCF binding prediction. 

In [ ]:
my_cnn1d = CNN_1d()
print(my_cnn1d.seq_len)
my_cnn1d
my_cnn1d, train_accs, val_accs = train_model(my_cnn1d, train_data, validation_data)

## Question 1 [4 points]

a. Find settings of `CNN_1d` that underfit (low train and test accuracy) and plot the train and validation accuracy. [3 points]

In [ ]:
# YOUR CODE HERE

b. Describe the setting choices you have made to underfit the data and explain why these settings contributed to a low train and test accuracy. [1 point]

*Fill in with your explanation. Feel free to add any plots or tables if you feel they will be helpful.*

## Question 2 [4 points]

a. Find settings of `CNN_1d` that that overfit (high train accuracy but low test accuracy). [3 points]

In [ ]:
# YOUR CODE HERE

b. Describe the setting choices you have made to overfit the data and explain why these settings contributed to a high train and low test accuracy. [1 point]

*Fill in with your explanation. Feel free to add any plots or tables if you feel they will be helpful.*

## Question 3  [6 points]

a. Carefully explore varying one architectural choice (e.g.  depth, number of channels, filter width, regularization, pooling factor, optimizer, learning rate, batch size, activation function, normalization, or skip connections). Report the final train and validation accuracy as a function of this choice. 

For example you might vary the number of convolutional layers from 1 to 4, keeping everything else the same, and plot validation accuracy vs number of layers. [4 points]

In [ ]:
# YOUR CODE HERE

b. For your selected architectural choice, discuss how and why varying this option affected your training and validation accuracy. [2 points]

*Fill in with your explanation. Feel free to add any plots or tables if you feel they will be helpful.*



## Question 4 [8 points]

a. Get as good validation accuracy as you can! [4 points]

Optionally you can try some more advanced extensions, e.g. 
1. Adding some [Recurrent Layers](https://pytorch.org/docs/stable/nn.html#recurrent-layers). Be warned that `torch` assumes the opposite dimensions for convolutional vs recurrent layers so you'll want to use `torch.transpose` appropriately. 
2. Transcription factors can bind to either strand of the DNA so you might want to include the [reverse complement](https://www.bx.psu.edu/old/courses/bx-fall08/definitions.html) (RC) in addition to the normal input. One suggestion for how to do this: run a copy of the network on the RC and take the max of the output from the two networks. Fancier approach here: https://www.biorxiv.org/content/10.1101/103663v1
3. Changing `BedPeaksDataset` to generate more negative examples in the space between peaks (although you'd only want to do this on the training data so you keep a fixed validation set). You will want to "cancel out" the additional negative examples by downweighting them in the loss. 

In [ ]:
# TODO

b. What gave you the biggest boost in performance? Why do you think that is? [2 points]

*Fill in with your explanation. Feel free to add any plots or tables if you feel they will be helpful.*

c.  Use some model interpretability technique to visualize why the model has made the assignments it did for a few examples. This can be one of the methods shown above (in silico mutagenesis or saliency maps) or one of the methods linked under *Other interpretation approaches*. [2 points]

In [ ]:
# TODO

## Question 5  [6 points]

The final task is to submit your best performing model's predictions on chromosomes 1 and 2 to our class competition. 

You can change what train/validation split you use here also if you want: e.g. you could do K-fold cross-validation or even retrain including the validation data if you think it will help (although early stopping may not work any more). 

First we'll need to load the test regions: 

In [ ]:
test_data = pd.read_csv(DATADIR + "ATAC_test_regions.bed.gz", sep='\t', names=("chrom","start","end"))
test_data.head(10)

Unlike the training data we've included random (in number and in genomic position) negative (no binding) regions in this bed file since otherwise you'd know implicitly that all the loaded regions are positives! 

We'll use a new `Dataset` class and `DataLoader` to match predictions on the test data in batches: 

In [ ]:
class BedPeaksDatasetTest(torch.utils.data.IterableDataset):
    
    def __init__(self, atac_data, genome, context_length):
        super(BedPeaksDatasetTest, self).__init__()
        self.context_length = context_length
        self.atac_data = atac_data
        self.genome = genome

    def __iter__(self): 
        for row in self.atac_data.itertuples():
            midpoint = int(.5 * (row.start + row.end))
            seq = self.genome[row.chrom][ midpoint - self.context_length//2:midpoint + self.context_length//2]
            yield(one_hot(seq))

test_dataset = BedPeaksDatasetTest(test_data, genome, my_cnn1d.seq_len) 
# you can always use a smaller batchsize if you ended up using a really big model
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, num_workers = 0) 

outputs = []
for x in test_dataloader: # iterate over batches
    x = x.to(device)
    output = my_cnn1d(x).squeeze() # your awesome model here! 
    output = torch.sigmoid(output)
    output_np = output.detach().cpu().numpy()
    outputs.append(output_np)
output_np = np.concatenate(outputs)

predicted_values=output_np.tolist() # Create list of your predictions from test data
pd.DataFrame(output_np).to_csv("path/to/file.csv") # Save to csv file
predicted_values[0:5]

Now you can make a submission to our class competition leaderboard! 

1. Sign up for a username and password at www.modelshare.org/login. 
2. Make sure your username is your Columbia Uni, so we know which submissions are yours. 
3. Use the below code to submit your predictions to the class [leaderboard](https://www.modelshare.org/detail/model:1484) ***(click compete tab)***

For the assignment you only need to make one leaderboard submission, but you can upload two submissions per day until the deadline if you want! 

Submission is worth 2 points, beating our baseline (auPR=0.82369) gets you 2 points and the other 2 points are for your tertile in the class (e.g. you get 2 points if you're in the top third, 1 point if you're in the middle third). 

In [ ]:
#install aimodelshare library
! pip install aimodelshare --upgrade

In [3]:
#Set credentials using modelshare.org username/password
from aimodelshare.aws import set_credentials

#Enter your modelshare.org username and password
apiurl="https://yro0pg9wja.execute-api.us-east-2.amazonaws.com/prod/m"
set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [4]:
#Instantiate Competition
import aimodelshare as ai
mycompetition= ai.Competition(apiurl)

In [12]:
#Submit Model predictions to leaderboard (without extracting model architecture information): 

# Submit Model 1 predictions to Competition Leaderboard
mycompetition.submit_model(model_filepath = None,
                                 preprocessor_filepath=None,
                                 prediction_submission=predicted_values)

# Repeat with new predictions to submit again!

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 6




In [13]:
# Get leaderboard to check your performance

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,auPR,ml_framework,model_type,username,version
0,49.47%,48.19%,49.31%,49.24%,0.502448,unknown,unknown,newusertest,5
1,49.47%,48.19%,49.31%,49.24%,0.502448,unknown,unknown,newusertest,6
2,49.97%,48.67%,49.77%,49.74%,nan,unknown,unknown,newusertest,1
3,50.31%,48.97%,50.01%,50.01%,nan,unknown,unknown,mikedparrott,2
4,49.97%,48.67%,49.77%,49.74%,nan,unknown,unknown,newusertest,3
5,49.97%,48.67%,49.77%,49.74%,nan,unknown,unknown,newusertest,4



**(Optional Extension) Submit Model With Custom Metadata:**

Can use to add team names or any other missing data you may wish to share on the leaderboard


In [ ]:
# Custom metadata can be added by passing a dict to the custom_metadata argument of the submit_model() method
# This option can be used to fill in missing data points or add new columns to the leaderboard

custom_meta = {'team': 'team one',
               'model_type': 'your_model_type',
               'new_column': 'new metadata'}

mycompetition.submit_model(model_filepath = None,
                                 preprocessor_filepath=None,
                                 prediction_submission=predicted_values,
                                 custom_metadata = custom_meta)

